In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 34kB/s 
     |████████████████████████████████| 3.8MB 32.0MB/s 
     |████████████████████████████████| 512kB 46.9MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=905ef9bdcfb44d14b4c4db5ea7c7fc165e2db83ad24b0d574c6a9a61aa039c7b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [3]:
import numpy as np

In [4]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [5]:
def building_ner(num_hidden_node, dropout, time_step, vector_length, output_lenght):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(25, time_step, vector_length)))

    model.add(TimeDistributed (LSTM(units=num_hidden_node, return_sequences=False, dropout=dropout, recurrent_dropout=dropout)) )

    model.add( Bidirectional(LSTM(units=num_hidden_node, return_sequences=True, dropout=dropout, recurrent_dropout=dropout), 
                             merge_mode='concat') )

    model.add(TimeDistributed(Dense(output_lenght)))
    
    model.add(Activation('softmax'))

    # plot_model(model, "demo.png", show_shapes=True)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = building_ner(num_hidden_node = 32, dropout = 0.2, time_step = 25, vector_length = 137, output_lenght = 384)
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 25, 25, 137)       0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 25, 32)            21760     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 64)            16640     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 25, 384)           24960     
_________________________________________________________________
activation_1 (Activation)    (None, 25, 384)           0         
Total params: 63,360
Trainable params

In [ ]:
tag = np.loadtxt('/content/drive/MyDrive/doanh/tag_embedd.txt')
print(tag.shape)
char_encode = np.loadtxt('/content/drive/MyDrive/doanh/char_encode.txt')
print(char_encode.shape)

(2500, 384)
(2500, 25)


In [ ]:
tag = tag.reshape(int(tag.shape[0]/25), 25, tag.shape[1])
print(tag.shape)
#new
char_encode = char_encode.reshape(int(char_encode.shape[0]/25), 25, char_encode.shape[1])
print(char_encode.shape)

(100, 25, 384)
(100, 25, 25)


In [ ]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

(100, 25, 25, 137)


In [ ]:
import keras

es_callback = keras.callbacks.EarlyStopping(monitor='acc', patience=5)

X_train, X_test, y_train, y_test = train_test_split(char_embedd, tag, test_size=0.1)

# Train the model with the new callback
model.fit(X_train, 
          y_train,
          epochs=100,
          batch_size = 64,
          validation_data=(X_test, y_test),
          callbacks=[es_callback])  # Pass callback to training




Train on 90 samples, validate on 10 samples
Epoch 1/100





90/90 [==============================] - 6s 69ms/step - loss: 5.9399 - acc: 0.0071 - val_loss: 5.9131 - val_acc: 0.0200
Epoch 2/100
90/90 [==============================] - 1s 9ms/step - loss: 5.9143 - acc: 0.1133 - val_loss: 5.8832 - val_acc: 0.5200
Epoch 3/100
90/90 [==============================] - 1s 8ms/step - loss: 5.8881 - acc: 0.3929 - val_loss: 5.8491 - val_acc: 0.5200
Epoch 4/100
90/90 [==============================] - 1s 8ms/step - loss: 5.8542 - acc: 0.4644 - val_loss: 5.8081 - val_acc: 0.5200
Epoch 5/100
90/90 [==============================] - 1s 8ms/step - loss: 5.8190 - acc: 0.4724 - val_loss: 5.7568 - val_acc: 0.5200
Epoch 6/100
90/90 [==============================] - 1s 8ms/step - loss: 5.7688 - acc: 0.4733 - val_loss: 5.6899 - val_acc: 0.5200
Epoch 7/100
90/90 [==============================] - 1s 8ms/step - loss: 5.7040 - acc: 0.4738 - val_loss: 5.6010 - val_acc: 0.5200
Epoch 8/100
90/90 [===========

In [ ]:
model.save("/content/drive/MyDrive/doanh/doanh.h5")